In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MyDrive/yob2010-2020.txt"
filename = f'/content/gdrive/MyDrive/name_classifier_RFC.sav'

Mounted at /content/gdrive


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import re
import spacy
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
def NER_names(text):
  token=sent_tokenize(text)
  
  names=[]
  for i in token:
    wordsList = nltk.word_tokenize(i)
    wordsList = [w for w in wordsList if not w in stop_words]
    tagged = nltk.pos_tag(wordsList)
    for words in tagged:
      if words[1]=='NNP':
        names.append(words[0])
  names=list(set(names))
  return names

In [14]:
def name_count(name):
  
    arr = np.zeros(52+26*26+3)
    # Iterate each character
    for ind, x in enumerate(name):
        arr[ord(x)-ord('a')] += 1
        arr[ord(x)-ord('a')+26] += ind+1
    # Iterate every 2 characters
    for x in range(len(name)-1):
        ind = (ord(name[x])-ord('a'))*26 + (ord(name[x+1])-ord('a')) + 52
        arr[ind] += 1
    # Last character
    arr[-3] = ord(name[-1])-ord('a')+1
    # Second Last character
    arr[-2] = ord(name[-2])-ord('a')+1
    # Length of name
    arr[-1] = len(name)
    return arr

In [ ]:
## Training function ## Ignore for inference
'''
my_data = np.genfromtxt(root_dir, 
                        delimiter=',', 
                        dtype=[('name','S50'), ('gender','S1'),('count','i6')],
                        converters={0: lambda s:s.lower()})
my_data = np.array([row for row in my_data if row[2]>=20])
name_map = np.vectorize(name_count, otypes=[np.ndarray])
my_data_1 = []
for i in range(0, len(my_data['name'])):
  my_data_1.append(my_data['name'][i].decode('UTF-8'))
Xlist = name_map(my_data_1)
X = np.array(Xlist.tolist())
y = my_data['gender']

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.33)

clf = RandomForestClassifier(n_estimators=100, min_samples_split=2)
clf.fit(Xtr, ytr)
pickle.dump(clf, open(filename, 'wb'))
'''

In [ ]:
## Accuracy function ## Ignore for inference
'''
def calc_accuracy(clf, Xtr, Xte, ytr, yte):
  y_pred=clf.predict(Xte)
  print(confusion_matrix(yte,y_pred))
  print(classification_report(yte,y_pred))
  print(accuracy_score(yte, y_pred))
clf= pickle.load(open(filename, 'rb'))
calc_accuracy(clf, Xtr, Xte, ytr, yte)
'''

"\ndef calc_accuracy(clf, Xtr, Xte, ytr, yte):\n  y_pred=clf.predict(Xte)\n  print(confusion_matrix(yte,y_pred))\n  print(classification_report(yte,y_pred))\n  print(accuracy_score(yte, y_pred))\nclf= pickle.load(open(filename, 'rb'))\ncalc_accuracy(clf, Xtr, Xte, ytr, yte)\n"

In [15]:
def NER_Pronouns(text):

  #print("Finding pronouns in ", text)

  nlp = spacy.load('en_core_web_sm')
  sent = text
  doc=nlp(sent)
  sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ] 
  sub_toks_pro = []
  for tok in doc:
    if tok.dep_ == "nsubj":
        if tok.pos_ == "PRON":
            sub_toks_pro.append(str(tok))
  return sub_toks_pro

In [16]:
def clf_pronoun(text):
  neutral_stopwords=['they', 'ours', 'your', 'themselves', 'ourselves', 'whom', 'them', 'their','theirs', 'those', 'these', 'yours', "you're", "you've", 'there', 'They', 'Ours', 'Your', 'Themselves', 'Ourselves', 'Whom', 'Them', 'Their', 'Theirs', 'Those', 'These', 'Yours', "You're", "You've", 'There']
  female_stopwords = ['she', 'her', 'hers', 'herself', 'She','Her','Herself', 'Hers']
  male_stopwords = ['he','him', 'his', 'himself', 'He', 'Him', 'His', 'Himself']
  
  words=text.lower().split(' ')
  if len(words)>1:

    pronoun_list = NER_Pronouns(text)

    if not pronoun_list:
      return ['N', 1]

    for pronoun in pronoun_list:
      if pronoun in female_stopwords:
        return ['F', 1]
      elif pronoun in male_stopwords:
        return ['M', 1]
      
    return ['N', 1]

  else:
    pronoun = words[0]
    if pronoun in female_stopwords:
        return ['F', 1]
    elif pronoun in male_stopwords:
        return ['M', 1]
    else:
        return ['N', 1]

In [17]:
clf= pickle.load(open(filename, 'rb'))

In [18]:
def gos_classifier(txt):

  txt = re.sub(r"[^a-zA-Z?!.,\s]","",txt)

  names=NER_names(txt)

  if len(names)>0:
    name_map = np.vectorize(name_count, otypes=[np.ndarray])
    predicted_gender=clf.predict(name_map(names).tolist())
    predicted_gender=predicted_gender[0].decode('UTF-8')
    temp=clf.predict_proba(name_map(names).tolist())
    predicted_probability=np.amax(temp)
    return predicted_gender,predicted_probability
  else:
    predicted_gender, predicted_probability= clf_pronoun(txt)
    return predicted_gender,predicted_probability

In [19]:
txt='The teacher, who is a professor at the university, says she was "gracious" as she was the only one to give the student a good'
gos_classifier(txt)

('F', 1)

In [20]:
# Read Dataset

import pandas as pd
file_path = "/content/gdrive/MyDrive/data.csv"
df = pd.read_csv(file_path)
df.head()

,Professions,GT
0,teacher,N
1,pathologist,N
2,technician,M
3,assistant,N
4,hairdresser,F


In [21]:
!pip3 install transformers
from transformers import pipeline, set_seed
from transformers import AutoTokenizer, AutoModelWithLMHead

     |████████████████████████████████| 3.3 MB 7.6 MB/s 
     |████████████████████████████████| 895 kB 56.8 MB/s 
     |████████████████████████████████| 596 kB 44.4 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
     |████████████████████████████████| 61 kB 696 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model_t5 = AutoModelWithLMHead.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:746: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [30]:
unmasker = pipeline('fill-mask', model="bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [31]:
def format_model_input(model_name,profession):
  if model_name=="bert-base-uncased":
    return "[MASK] is a {}.".format(profession)
  
  elif model_name=="roberta-base":
    return "<mask> is a {}.".format(profession)

  elif model_name=="gpt2":     
    return "The {},".format(profession)
  
  elif model_name=="t5-base":
    return 'summarize: The {},'.format(profession)

def model(model_name,model_input):

  # This function return model output for a given model and it's input
  # It uses postprocess function to format the output for logs

  # For masked model: model_output and classifier score
  # For unmasked model: model_output

  if model_name=="bert-base-uncased":
    return postprocess(model_name,unmasker(model_input))
  
  elif model_name=="roberta-base":
    #unmasker = pipeline('fill-mask', model=model_name)
    return postprocess(model_name,unmasker(model_input))

  elif model_name=="gpt2":     
    generator = pipeline('text-generation', model = model_name )
    #set_seed(42)
    return postprocess(model_name,generator(model_input, max_length=30, num_return_sequences=1))
  
  elif model_name=="t5-base":
    # set_seed(41)
    
    input_ids = tokenizer.encode(model_input, return_tensors='pt')
    greedy_output = model_t5.generate(input_ids, num_beams=7, no_repeat_ngram_size=2, min_length=50, max_length=100)
    return postprocess(model_name, tokenizer.decode(greedy_output[0], skip_special_tokens=True))

In [23]:
def postprocess(model_name,model_output):

  if model_name=="bert-base-uncased":
    return model_output[0]["token_str"], model_output[0]["score"]
  
  elif model_name=="roberta-base":
    return [model_output[i]["token_str"] for i in range(1)][0],[model_output[i]["score"] for i in range(1)][0]

  elif model_name=="gpt2":
    return model_output[0]["generated_text"]
  
  elif model_name=="t5-base":
    return model_output

In [24]:
def generate_result(results, scores):
  
  # if results==None or scores==None:
  #   return "NA", 0

  # if type(results) is not list or type(scores) is not list:
  #   return "NA", 0

  values = {"M" : 0, "F" : 0, "N" : 0}
  scores_dict = {"M" : 0, "F" : 0, "N" : 0}

  for i in range(len(results)):
    values[results[i]] += 1
    scores_dict[results[i]] += scores[i]
  
  for i in scores_dict:
    if values[i]!=0:
      scores_dict[i] = scores_dict[i] / values[i]

  if values["M"] > values["N"] and values["M"] > values["F"]:
    return "M", scores_dict["M"]
  
  elif values["F"] > values["N"] and values["F"] > values["M"]:
    return "F", scores_dict["F"]
  
  elif values["N"] > values["M"] and values["N"] > values["F"]:
    return "N", scores_dict["N"]

  elif values["N"] == values["M"] and values["M"] > values["F"]:
    return "M", scores_dict["M"]
  
  elif values["N"] == values["F"] and values["F"] > values["M"]:
    return "F", scores_dict["F"]

  else:
    return "N", scores_dict["N"]

In [25]:
def merge_csv_files():

  df_merged = pd.DataFrame()
  df_merged["Professions"] = df["Professions"]
  df_merged["Ground Truth"] = df["GT"]

  for model in models:
    file_name = model + ".csv"
    df_model_results = pd.read_csv(file_name)
    df_merged[model] = df_model_results["Results"]
  
  df_merged.to_csv("results.csv")

  return df_merged.head()

In [ ]:
# For each profession in dataset, call each model one by one
# For masked model, call them once
# For unmasked model, call them 10 times

def test_model(model_name,iterations,masked):
  file_name=model_name+".txt"
  logs={}

  with open(file_name,"w+") as log_file:
    
    for profession in df["Professions"]:

      logs[profession] = {}
      df_dict["Professions"].append(profession)

      model_input = format_model_input(model_name, profession)
      predictions = []
      prediction_scores = []
      
      for iteration in range(iterations):

        print(model_name, profession, iteration)

        if masked:

          model_output, classifier_score = model( model_name, model_input )
          classifier_output, gos_score = gos_classifier ( model_output )
          logs[profession][iteration] = {
          "Input": model_input,
          "Output": model_output,
          "GOS": classifier_output,
          "Score": classifier_score
          }

        else:
          model_output = model ( model_name, model_input )

          try:
            classifier_output, classifier_score = gos_classifier ( model_output )
          except:
            pass
          logs[profession][iteration] = {
            "Input": model_input,
            "Output": model_output,
            "GOS": classifier_output,
            "GOS Score": classifier_score
          }

        predictions.append(classifier_output)
        prediction_scores.append(classifier_score)


      pred_result, pred_score = generate_result(predictions, prediction_scores)
      df_dict["Results"].append(pred_result)
      df_dict["Scores"].append(pred_score)
    

    # Append current log to csv
    log_file.write(str(logs))
    df_csv = pd.DataFrame(df_dict)
    csv_name = model_name + ".csv"
    df_csv.to_csv(csv_name, index = False)
  
# Run models
models=["bert-base-uncased","roberta-base","t5-base","gpt2"]

df_dict = {"Professions":[], "Results":[], "Scores":[]}
test_model(models[0],1,True)

df_dict = {"Professions":[], "Results":[], "Scores":[]}
test_model(models[1],1,True)

df_dict = {"Professions":[], "Results":[], "Scores":[]}
test_model(models[2],10,False)

df_dict = {"Professions":[], "Results":[], "Scores":[]}
test_model(models[3],10,False)

merge_csv_files()